In [1]:
from artifacts_ipv6_sra_scanning.config import *

2025-10-28 22:58:39.504 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
df_scan01 = pl.read_csv(f'{RAW_DATA_DIR}/bgp48_1728075185_routerIP_stats.csv',separator=';')
df_scan01 = df_scan01.rename({"ASN": "AS-Number",'numamplsubnets_48':'numamplsubnets','numsubnets_48':'numsubnets','Prefix':'BGP-Prefix','routerip':'saddr'})

In [3]:
pd_df = df_scan01.filter((pl.col('numamplsubnets')>0) & (pl.col('classification')=='timxceed')).sort('maxampl',descending=True).to_pandas()
loop_df = df_scan01.filter((pl.col('classification')=='timxceed')).sort('numsubnets',descending=True).to_pandas()

# Table 4a

In [4]:
totalloops = df_scan01.filter((pl.col('classification')=='timxceed'))['numsubnets'].sum()
#print(
df_scan01.filter((pl.col('classification')=='timxceed') & (pl.col('Geo').is_not_null())).group_by(['Geo']).agg(
    pl.col('numsubnets').sum().alias('routingloops'),
    (pl.col('numsubnets').sum()/totalloops).mul(100).alias('relative'),
    pl.col('saddr').n_unique().alias('routerips'),
).top_k(5,by='routingloops').to_pandas()#.to_latex(index=False,float_format="%.2f"))

,Geo,routingloops,relative,routerips
0,BRA,37081970,26.222086,9329
1,DEU,13273666,9.386319,1192
2,CZE,10444197,7.385493,881
3,USA,7613551,5.383835,4150
4,NLD,7241713,5.120894,388


In [5]:
totalloops = df_scan01.filter((pl.col('classification')=='timxceed'))['numsubnets'].sum()
df_scan01.filter((pl.col('classification')=='timxceed') & (pl.col('Geo').is_not_null())).group_by(['Geo']).agg(
    pl.col('numsubnets').sum().alias('routingloops'),
    (pl.col('numsubnets').sum()/totalloops).mul(100).alias('relative'),
    pl.col('saddr').n_unique().alias('routerips'),
).top_k(5,by='routingloops')['relative'].sum()

53.49862680080404

# Table 4b

In [6]:
totalamp = df_scan01.filter((pl.col('classification')=='timxceed') & (pl.col('numamplsubnets')>0))['numsubnets'].sum()
#print(
df_scan01.filter((pl.col('classification')=='timxceed') & (pl.col('Geo').is_not_null()) & (pl.col('numamplsubnets')>0)).group_by(['Geo']).agg(
    pl.col('numamplsubnets').sum().alias('amplification'),
    (pl.col('numamplsubnets').sum()/totalamp).mul(100).alias('relative'),
    pl.col('saddr').n_unique().alias('routerips'),
    pl.col('maxampl').max().alias('maxamplification'),
).top_k(5,by='amplification').to_pandas()#.to_latex(index=False,float_format="%.2f"))

,Geo,amplification,relative,routerips,maxamplification
0,BRA,4674687,23.351353,2765,51
1,DEU,477046,2.382977,78,258399
2,USA,247736,1.237510,577,109385
3,CHN,245546,1.226570,4141,52
4,CZE,212547,1.061731,17,37


In [7]:
loop_df['AS-Number'].nunique()

5531

In [8]:
len(loop_df[loop_df.numsubnets==1])/len(loop_df)

0.604160816250117